<a href="https://colab.research.google.com/github/jmmrcp/TFM/blob/main/Dataset_Evaluado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re

In [ ]:
# Estadísticas basadas en análisis previos
LONGITUD_MEDIA_PHISHING = 150
LONGITUD_MEDIA_LEGITIMO = 500
LONGITUD_MINIMA_LEGITIMO = 300
LONGITUD_TOLERANCIA = 100

In [ ]:
def limpiar_mensaje(mensaje):
    """
    Limpia el mensaje de caracteres no deseados y extraños.
    :param mensaje: El contenido del correo a limpiar.
    :return: Mensaje limpio.
    """
    # Eliminar caracteres no deseados
    mensaje = re.sub(r'[^\x00-\x7F]+', '', mensaje)  # Elimina caracteres fuera del rango ASCII
    mensaje = mensaje.replace('\n', ' ').replace('\r', '')  # Reemplaza saltos de línea
    return mensaje.strip()  # Elimina espacios en blanco al principio y al final

In [ ]:
def seleccionar_mejores_correos(df, num_mejores=500):
    """
    Selecciona los mejores correos de phishing y legítimos y ajusta las etiquetas.

    :param df: DataFrame que contiene los correos.
    :param num_mejores: Número de correos a seleccionar para cada categoría.
    :return: DataFrame con los mejores correos seleccionados y etiquetas ajustadas.
    """
    # Filtrar correos de phishing y legítimos
    phishing_emails = df[df['label'] == 1].copy()
    legitimate_emails = df[df['label'] == 0].copy()

    # Ordenar por probabilidad (de mayor a menor)
    phishing_emails.sort_values(by='probabilidad', ascending=False, inplace=True)
    legitimate_emails.sort_values(by='probabilidad', ascending=False, inplace=True)

    # Seleccionar los mejores 400 de cada categoría
    mejores_phishing = phishing_emails.head(num_mejores)
    mejores_legitimos = legitimate_emails.head(num_mejores)

    # Ajustar las etiquetas usando .loc para evitar SettingWithCopyWarning
    mejores_phishing.loc[:, 'label'] = 'phishing'
    mejores_legitimos.loc[:, 'label'] = 'legitimate'

    # Combinar los dos DataFrames
    mejores_correos = pd.concat([mejores_phishing, mejores_legitimos], ignore_index=True)

    # Mezclar los registros aleatoriamente
    mejores_correos = mejores_correos.sample(frac=1).reset_index(drop=True)

    # Seleccionar solo las columnas 'label' y 'message'
    mejores_correos = mejores_correos[['label', 'message']]

    return mejores_correos

In [ ]:
def es_calidad_aceptable(mensaje, tipo):
    # Validación simple basada en longitud y palabras clave
    if not isinstance(mensaje, str):
        return False

    longitud = len(mensaje)

    if tipo == 1:  # Phishing
        phishing_keywords = ["urgent", "password", "account", "click here", "verify", "bank", "login", "suspend", "update", "limited time"]
        if not any(keyword in mensaje.lower() for keyword in phishing_keywords):
            return False
        if abs(longitud - LONGITUD_MEDIA_PHISHING) > LONGITUD_TOLERANCIA:
            return False

    elif tipo == 0:  # Legitimate
        legitimate_keywords = ["meeting", "schedule", "project", "report", "invoice", "team", "update", "conference", "client", "feedback", "contract", "deadline", "greetings", "thank you", "best regards", "regards"]
        if not any(keyword in mensaje.lower() for keyword in legitimate_keywords):
            return False
        if abs(longitud - LONGITUD_MEDIA_LEGITIMO) > LONGITUD_TOLERANCIA:
            return False
        if longitud < LONGITUD_MINIMA_LEGITIMO:
            return False
        estructura_legitima = ["regards", "thank you", "dear", "sincerely", "best regards", "kind regards"]
        if not any(estructura in mensaje.lower() for estructura in estructura_legitima):
            return False
    return True

In [ ]:
def entrenar_modelo(df):
    """
    Entrena un modelo RandomForest con el dataset disponible.
    :param df: DataFrame con columnas 'message' y 'label'.
    :return: Modelo entrenado (RandomForestClassifier).
    """
    # Convertir textos en características simples basadas en longitud y presencia de palabras clave
    df['longitud'] = df['message'].apply(len)
    df['num_phishing_keywords'] = df['message'].apply(lambda msg: sum([1 for word in ["urgent", "password", "account", "click here"] if word in msg.lower()]))
    df['num_legitimate_keywords'] = df['message'].apply(lambda msg: sum([1 for word in ["meeting", "schedule", "thank you", "regards"] if word in msg.lower()]))

    # Preparar características y etiquetas
    X = df[['longitud', 'num_phishing_keywords', 'num_legitimate_keywords']]
    y = df['label']

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar un modelo Random Forest
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)

    return modelo

In [ ]:
def extraer_mejores_correos(df, modelo, max_por_clase=400):
    """
    Extrae los mejores correos aplicando aleatoriedad y validación con el modelo de clasificación.
    :param df: DataFrame con columnas 'message' y 'label'.
    :param modelo: Modelo de clasificación entrenado.
    :param max_por_clase: Máximo número de correos a seleccionar por clase.
    :return: DataFrame con correos balanceados y validados.
    """
    # Filtrar los correos legítimos (label == 0)
    legitimate_emails = df[df['label'] == 0]
    phishing_emails = df[df['label'] == 1]

    # Agregar predicciones de probabilidad del modelo
    legitimate_emails['probabilidad'] = modelo.predict_proba(legitimate_emails[['longitud', 'num_phishing_keywords', 'num_legitimate_keywords']])[:, 1]
    phishing_emails['probabilidad'] = modelo.predict_proba(phishing_emails[['longitud', 'num_phishing_keywords', 'num_legitimate_keywords']])[:, 1]

    # Selección aleatoria controlada (50% aleatorio y 50% basado en probabilidad alta)
    legitimate_random = legitimate_emails.sample(frac=0.5)
    legitimate_high_quality = legitimate_emails.sort_values(by='probabilidad', ascending=False).head(max_por_clase)

    phishing_random = phishing_emails.sample(frac=0.5)
    phishing_high_quality = phishing_emails.sort_values(by='probabilidad', ascending=False).head(max_por_clase)

    # Unir las selecciones y mezclarlas
    dataset_balanceado = pd.concat([legitimate_random, legitimate_high_quality, phishing_random, phishing_high_quality]).sample(frac=1).reset_index(drop=True)

    return dataset_balanceado

In [ ]:
# Cargar el DataFrame con columnas 'message' y 'label'
df = pd.read_csv('/content/drive/MyDrive/DATA/datasets/full.csv')  # Cargar los datos desde un archivo CSV

In [ ]:
# Entrenar el modelo
modelo = entrenar_modelo(df)

In [ ]:
# Extraer los mejores correos aplicando aleatoriedad y validación del modelo
dataset_balanceado = extraer_mejores_correos(df, modelo, max_por_clase=400)

In [ ]:
# Limpiar los mensajes antes de la validación
dataset_balanceado['message'] = dataset_balanceado['message'].apply(limpiar_mensaje)

In [ ]:
# Mostrar el resultado
print(dataset_balanceado)

In [ ]:
mejores_correos = seleccionar_mejores_correos(dataset_balanceado)

In [ ]:
# Mostrar el resultado
print(mejores_correos)

In [ ]:
# Guardar el DataFrame resultante en un archivo CSV
mejores_correos.to_csv('/content/drive/MyDrive/DATA/datasets/Best-1k.csv', index=False)